# Clustering Crypto

In [75]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [76]:
# Load the crypto_data.csv dataset.
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [77]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df["IsTrading"] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [78]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns=["IsTrading"])
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [79]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
print(crypto_df)

         CoinName    Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42        42 Coin       Scrypt   PoW/PoS     4.199995e+01              42
404       404Coin       Scrypt   PoW/PoS     1.055185e+09       532000000
808           808      SHA-256   PoW/PoS     0.000000e+00               0
1337    EliteCoin          X13   PoW/PoS     2.927942e+10    314159265359
BTC       Bitcoin      SHA-256       PoW     1.792718e+07        21000000
...           ...          ...       ...              ...             ...
ZEPH       ZEPHYR      SHA-256      DPoS     2.000000e+09      2000000000
GAP       Gapcoin       Scrypt   PoW/PoS     1.493105e+07       250000000
BDX        Beldex  CryptoNight       PoW     9.802226e+08      1400222610
ZEN       Horizen     Equihash       PoW     7.296538e+06        21000000
XBC   BitcoinPlus       Scrypt       PoS     1.283270e+05         1000000

[685 rows x 5 columns]


In [80]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]
print(crypto_df)

         CoinName    Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42        42 Coin       Scrypt   PoW/PoS     4.199995e+01              42
404       404Coin       Scrypt   PoW/PoS     1.055185e+09       532000000
1337    EliteCoin          X13   PoW/PoS     2.927942e+10    314159265359
BTC       Bitcoin      SHA-256       PoW     1.792718e+07        21000000
ETH      Ethereum       Ethash       PoW     1.076842e+08               0
...           ...          ...       ...              ...             ...
ZEPH       ZEPHYR      SHA-256      DPoS     2.000000e+09      2000000000
GAP       Gapcoin       Scrypt   PoW/PoS     1.493105e+07       250000000
BDX        Beldex  CryptoNight       PoW     9.802226e+08      1400222610
ZEN       Horizen     Equihash       PoW     7.296538e+06        21000000
XBC   BitcoinPlus       Scrypt       PoS     1.283270e+05         1000000

[532 rows x 5 columns]


In [81]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = crypto_df[["CoinName"]]
crypto_names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [82]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=['CoinName'], inplace = True)

In [83]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
X


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(X)
print(crypto_scaled[0:5])




[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [85]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
print(crypto_pca)

[[-0.33521849  1.02891247 -0.45667105]
 [-0.31856428  1.02914473 -0.45714373]
 [ 2.30469385  1.59792692 -0.60762134]
 ...
 [ 0.31632259 -2.33942931  0.35590954]
 [-0.1490354  -2.15637453  0.31291478]
 [-0.29697473  0.85293193 -0.18566789]]


In [86]:
# Create a DataFrame with the three principal components.
columns = ['PC 1', 'PC 2', 'PC 3']
pca_df = pd.DataFrame(data=crypto_pca, columns=columns, index=crypto_df.index)
pca_df.head()


,PC 1,PC 2,PC 3
42,-0.335218,1.028912,-0.456671
404,-0.318564,1.029145,-0.457144
1337,2.304694,1.597927,-0.607621
BTC,-0.140756,-1.300838,0.102861
ETH,-0.144551,-2.004619,0.288984


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [87]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values.
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)

    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot.
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")




:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [88]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)
print(predictions)


[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 0 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 0 3 3 0 0 3 0 0 3 3 0 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 1 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 0 0
 3 3 0 3 3 3 3 0 1 1 3 3 3 0 1 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 0 3 0 3 3 0 0 3 0 3 3 3 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

In [89]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatenate the crypto_df and crypto_pca_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pca_df], axis=1, sort=False)

# Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies.
clustered_df['CoinName'] = crypto_names_df['CoinName']

# Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)


(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.335218,1.028912,-0.456671,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.318564,1.029145,-0.457144,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.304694,1.597927,-0.607621,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.140756,-1.300838,0.102861,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.144551,-2.004619,0.288984,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.166760,-1.116259,0.010185,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.395853,1.217779,-0.413163,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.146890,-2.247461,0.332859,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.142994,-2.004716,0.288967,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.149034,-2.156374,0.312915,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [90]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x='PC 1',
    y='PC 2',
    z='PC 3',
    color='Class',
    hover_name='CoinName',
    hover_data=['Algorithm'],
    width=800,
    height=800
)

# Show the plot
fig.show()


In [91]:
# Create a table with tradable cryptocurrencies
table = clustered_df[['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']]

# Display the table in a Jupyter notebook
table_plot = table.hvplot.table(width=900, sortable=True, selectable=True)
table_plot


:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [92]:
# Print the total number of tradable cryptocurrencies
total_tradable_cryptocurrencies = table.shape[0]
print(f'Total number of tradable cryptocurrencies: {total_tradable_cryptocurrencies}')


Total number of tradable cryptocurrencies: 532


In [93]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(table[['TotalCoinsMined', 'TotalCoinSupply']])

In [94]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_df = pd.DataFrame(scaled_values, columns=['TotalCoinsMined_scaled', 'TotalCoinSupply_scaled'], index=table.index)


# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_df['CoinName'] = table['CoinName']


# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_df['Class'] = table['Class']

scaled_df

,TotalCoinsMined_scaled,TotalCoinSupply_scaled,CoinName,Class
42,0.000000e+00,4.200000e-11,42 Coin,0
404,1.065855e-03,5.320000e-04,404Coin,0
1337,2.957551e-02,3.141593e-01,EliteCoin,0
BTC,1.810842e-05,2.100000e-05,Bitcoin,3
ETH,1.087731e-04,0.000000e+00,Ethereum,3
...,...,...,...,...
ZEPH,2.020225e-03,2.000000e-03,ZEPHYR,0
GAP,1.508199e-05,2.500000e-04,Gapcoin,0
BDX,9.901351e-04,1.400223e-03,Beldex,3
ZEN,7.370282e-06,2.100000e-05,Horizen,3


In [95]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_df.hvplot.scatter(
    x='TotalCoinsMined_scaled',
    y='TotalCoinSupply_scaled',
    by='Class',
    hover_cols=['CoinName'],
    width=900,
    height=500
)



:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined_scaled]   (TotalCoinSupply_scaled,CoinName)